Note. Due to filesize, the raw data wasn't uploaded to this repository. The files can be found at: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews/code

In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import bz2
import re
import os
import gc
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from official.nlp import optimization  # to create AdamW optimizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evanmeeks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/evanmeeks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'official'

In [2]:
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')

In [3]:
def load_extract(file):
    texts, labels = [], [] 
    for line in file:
        x = line.decode('utf-8') 
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts

In [4]:
train_labels, train_text = load_extract(train_file)
test_labels, test_text = load_extract(test_file)

In [5]:
def clean_texts(texts):
    stwords = stopwords.words('english')
    temp_texts = []
    for i in range(len(texts)):
        text = re.sub('\d','0',texts[i])
        if 'www.' in text or 'http:' in text or 'https:' in text or '.com' in text: 
            text = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", " ", text)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = text.split()
        text = [word for word in text if not word in stwords]
        text = ' '.join(text)
        temp_texts.append(text)
    return temp_texts

In [6]:
train_texts = clean_texts(train_text)
test_texts = clean_texts(test_text)

In [7]:
def tokenize(text):
    x = []
    for line in text: 
        x += word_tokenize(line)
    return x

In [8]:
train_tokens = tokenize(train_text)
test_tokens = tokenize(test_text)

In [9]:
test_tokens

['Great',
 'CD',
 ':',
 'My',
 'lovely',
 'Pat',
 'has',
 'one',
 'of',
 'the',
 'GREAT',
 'voices',
 'of',
 'her',
 'generation',
 '.',
 'I',
 'have',
 'listened',
 'to',
 'this',
 'CD',
 'for',
 'YEARS',
 'and',
 'I',
 'still',
 'LOVE',
 'IT',
 '.',
 'When',
 'I',
 "'m",
 'in',
 'a',
 'good',
 'mood',
 'it',
 'makes',
 'me',
 'feel',
 'better',
 '.',
 'A',
 'bad',
 'mood',
 'just',
 'evaporates',
 'like',
 'sugar',
 'in',
 'the',
 'rain',
 '.',
 'This',
 'CD',
 'just',
 'oozes',
 'LIFE',
 '.',
 'Vocals',
 'are',
 'jusat',
 'STUUNNING',
 'and',
 'lyrics',
 'just',
 'kill',
 '.',
 'One',
 'of',
 'life',
 "'s",
 'hidden',
 'gems',
 '.',
 'This',
 'is',
 'a',
 'desert',
 'isle',
 'CD',
 'in',
 'my',
 'book',
 '.',
 'Why',
 'she',
 'never',
 'made',
 'it',
 'big',
 'is',
 'just',
 'beyond',
 'me',
 '.',
 'Everytime',
 'I',
 'play',
 'this',
 ',',
 'no',
 'matter',
 'black',
 ',',
 'white',
 ',',
 'young',
 ',',
 'old',
 ',',
 'male',
 ',',
 'female',
 'EVERYBODY',
 'says',
 'one',
 'thing

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(train_texts)

train_texts_vec = count_vect.transform(train_texts)

test_texts_vec = count_vect.transform(test_texts)


In [12]:
test_texts_vec

<400000x866686 sparse matrix of type '<class 'numpy.int64'>'
	with 13565406 stored elements in Compressed Sparse Row format>

In [13]:
nb = MultinomialNB()
nb.fit(train_texts_vec, train_labels)

MultinomialNB()

In [14]:
y_pred = nb.predict(test_texts_vec)

In [15]:
print('Accuracy:', accuracy_score(test_labels, y_pred))

Accuracy: 0.8491875


In [16]:
lr_model = LogisticRegression(n_jobs=-1, max_iter=150)
lr_model.fit(train_texts_vec, train_labels)

/Users/evanmeeks/opt/anaconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=150, n_jobs=-1)

In [17]:
pred_lr = lr_model.predict(test_texts_vec)
print('Accuracy:', accuracy_score(test_labels, pred_lr))

Accuracy: 0.9006075


In [18]:
sample = test_texts[100]
print(sample)

sample_vec = count_vect.transform([sample])
pred = lr_model.predict(sample_vec)
print('\npredicted label:',pred[0])
print('actual label:', test_labels[100])

problems munchkin version ditto complaints separate easily either unscrewing prizing cheap flimsy leaks water also gasket looking something else hoping reviewers give honest good advice almost identical product munchkin

predicted label: 0
actual label: 0
